### Ομάδα 70:
**Χρήστος Παπαδημητρίου, el18017**

**Ζαχαρίας-Παύλος Αναστασιάδης, el18161**

**Βαρθολομαίος Βαμβακάρης, el18071**

# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 
Ημερομηνία εκφώνησης της άσκησης: 29 Νοεμβρίου 2022

**Θα βρείτε το παρόν σε μορφή jupyter notebook ως συνημμένο στο τέλος της εκφώνησης.**


In [1]:
# !pip install --upgrade pip
!pip install numpy
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install joblib

## Εισαγωγή του Dataset

Το σύνολο δεδομένων με το οποίο θα δουλέψουμε είναι βασισμένο στο [Carnegie Mellon Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). Πρόκειται για ένα dataset με 22.301 περιγραφές ταινιών. Η περιγραφή κάθε ταινίας αποτελείται από τον τίτλο της, μια ή περισσότερες ετικέτες που χαρακτηρίζουν το είδος της ταινίας και τέλος τη σύνοψη της υπόθεσής της. Αρχικά εισάγουμε το dataset (χρησιμοποιήστε αυτούσιο τον κώδικα, δεν χρειάζεστε το αρχείο csv) στο dataframe `df_data_1`: 

In [2]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1zo13kUAf-MDMPZmBDxq1FxWtZY01lsxD"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Κάθε ομάδα θα δουλέψει σε **ένα μοναδικό υποσύνολο 5.000 ταινιών** (διαφορετικό dataset για κάθε ομάδα) ως εξής:

1. Κάθε ομάδα του εργαστηρίου νευρωνικών έχει έναν αριθμό στο helios. Θα βάλετε τον αριθμό αυτό στη μεταβλητή team_seed_number στο επόμενο κελί κώδικα.

2. Το data frame `df_data_2` έχει γραμμές όσες και οι ομάδες και 5.000 στήλες. Σε κάθε ομάδα αντιστοιχεί η γραμμή του πίνακα με το `team_seed_number` της. Η γραμμή αυτή θα περιλαμβάνει 5.000 διαφορετικούς αριθμούς που αντιστοιχούν σε ταινίες του αρχικού dataset. 

3. Τρέξτε τον κώδικα. Θα προκύψουν τα μοναδικά για κάθε ομάδα  titles, categories, catbins, summaries και corpus με τα οποία θα δουλέψετε.

In [3]:
import numpy as np

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 70

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1g6F4TCHrs2wgtdOk7D3gtONaeirNt_Vo"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus

- Ο πίνακας **titles** περιέχει τους τίτλους των ταινιών. Παράδειγμα: 'Sid and Nancy'.
- O πίνακας **categories** περιέχει τις κατηγορίες (είδη) της ταινίας υπό τη μορφή string. Παράδειγμα: '"Tragedy",  "Indie",  "Punk rock",  "Addiction Drama",  "Cult",  "Musical",  "Drama",  "Biopic \[feature\]",  "Romantic drama",  "Romance Film",  "Biographical film"'. Παρατηρούμε ότι είναι μια comma separated λίστα strings, με κάθε string να είναι μια κατηγορία.
- Ο πίνακας **catbins** περιλαμβάνει πάλι τις κατηγορίες των ταινιών αλλά σε δυαδική μορφή ([one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)). Έχει διαστάσεις 5.000 x 322 (όσες οι διαφορετικές κατηγορίες). Αν η ταινία ανήκει στο συγκεκριμένο είδος η αντίστοιχη στήλη παίρνει την τιμή 1, αλλιώς παίρνει την τιμή 0.
- Ο πίνακας **summaries** και η λίστα **corpus** περιλαμβάνουν τις συνόψεις των ταινιών (η corpus είναι απλά ο summaries σε μορφή λίστας). Κάθε σύνοψη είναι ένα (συνήθως μεγάλο) string. Παράδειγμα: *'The film is based on the real story of a Soviet Internal Troops soldier who killed his entire unit  as a result of Dedovschina. The plot unfolds mostly on board of the prisoner transport rail car guarded by a unit of paramilitary conscripts.'*
- το dataframe **corpus_df** που είναι απλά το corpus σε μορφή dataframe. Τα summaries βρίσκονται στην κολόνα 0. Πιθανώς να σας βολεύει να κάνετε κάποιες προεπεξεργασίες με dataframes.


Θεωρούμε ως **ID** της κάθε ταινίας τον αριθμό γραμμής της ή το αντίστοιχο στοιχείο της λίστας. Παράδειγμα: για να τυπώσουμε τη σύνοψη της ταινίας με `ID=999` (την χιλιοστή) θα γράψουμε `print(corpus[999])`.

In [4]:
ID = 999
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

['Eddelu Manjunatha']
['"Drama",  "Comedy"']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.

# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

Η πρώτη εφαρμογή που θα αναπτύξετε θα είναι ένα [σύστημα συστάσεων](https://en.wikipedia.org/wiki/Recommender_system) ταινιών βασισμένο στο περιεχόμενο (content based recommender system). Τα συστήματα συστάσεων στοχεύουν στο να προτείνουν αυτόματα στο χρήστη αντικείμενα από μια συλλογή τα οποία ιδανικά θέλουμε να βρει ενδιαφέροντα ο χρήστης. Η κατηγοριοποίηση των συστημάτων συστάσεων βασίζεται στο πώς γίνεται η επιλογή (filtering) των συστηνόμενων αντικειμένων. Οι δύο κύριες κατηγορίες είναι η συνεργατική διήθηση (collaborative filtering) όπου το σύστημα προτείνει στο χρήστη αντικείμενα που έχουν αξιολογηθεί θετικά από χρήστες που έχουν παρόμοιο με αυτόν ιστορικό αξιολογήσεων και η διήθηση με βάση το περιεχόμενο (content based filtering), όπου προτείνονται στο χρήστη αντικείμενα με παρόμοιο περιεχόμενο (με βάση κάποια χαρακτηριστικά) με αυτά που έχει προηγουμένως αξιολογήσει θετικά.

Το σύστημα συστάσεων που θα αναπτύξετε θα βασίζεται στο **περιεχόμενο** και συγκεκριμένα στις συνόψεις των ταινιών (corpus). 


## Προεπεξεργασία

Το πρώτο βήμα στην επεξεργασία μας είναι ο καθαρισμός των περιγραφών των ταινιών. 

Εκτυπώστε (αρκετές) διαφορετικές περιγραφές ταινιών για να δείτε πιθανά προβλήματα που θα πρέπει να αντιμετωπιστούν.

Τα (ελάχιστα) βήματα καθαρισμού που προτείνουμε είναι:
- μετατροπή όλων των χαρακτήρων σε πεζά,
- αφαίρεση των stopwords. Εδώ σημειώστε ότι για το δεδομένο task του συστήματος συστάσεων που είναι η πρόταση ταινιών ίσως θα είχαν ενδιαφέρον και λίστες stopwords πέραν αυτών της κοινής γλώσσας.
- αφαίρεση σημείων στίξης και ειδικών χαρακτρήρων (special characters). Αυτό δεν γίνεται μόνο με την punkt του NLTK. Θα μπορούσατε να βασιστείτε σε κανονικές εκφράσεις (regular expressions), και
- αφαίρεση πολυ σύντομων συμβολοσειρών.

Προσοχή: το corpus και τα τελικά tokens που θα το αποτελούν θα χρησιμοποιηθούν στη συνέχεια ως κλειδιά για να βρούμε εμφυτεύματα. Για το λόγο αυτό, πρέπει να είστε προσεκτικοί ως προς την εφαρμογή μεθόδων κανονικοποίησης (text normalization) όπως το stemming και το lemmatization.

In [5]:
import collections
import nltk
import string
from nltk.corpus import stopwords
import joblib
import random

**Μετατροπή όλων των χαρακτήρων σε πεζα**

In [6]:
def all2lowercase(text):
    return text.lower()

In [7]:
corpus = list(map(all2lowercase, corpus))

**Αφαίρεση Stopwords**

**Αφαίρεση Σημείων στίξης**

In [8]:
def checkExtraChars(word, chars):
    l = []
    final_word = ""
    for letter in word:
        l.append((letter in chars))
        final_word = final_word+letter if letter not in chars else final_word
    return final_word if not all(l) else None        

**Αφαίρεση πολύ σύντομων συμβολοσειρών**

In [9]:
def preprocess_document(document):
    # all words to lowercase
    words = nltk.word_tokenize(document.lower())

    # remove stopwords
    myStopWords = ["film", "movie", "story", "n't"]
    words_no_stopwords = [word for word in words if word not in stopwords.words('english')+myStopWords]
    
    # remove punctuation
    words_no_punctuation = [word for word in words_no_stopwords if word not in list(string.punctuation)]
    
    # replace some strings
    words_replaced =  list(map(lambda word: word.replace('.', ''), words_no_punctuation))
    
    # remove some special characters
    words_no_extra_chars, extraChars = [], ["'", "`", "\"", "-", "_"]
    for word in words_replaced:
        if checkExtraChars(word, extraChars) != None:
            words_no_extra_chars.append(checkExtraChars(word, extraChars))
            
    # remove urls (which we identify as strings with more than one '/')
    words_no_urls = []
    for word in words_no_extra_chars:
        if word.count('/') < 2:
            words_no_urls.append(word.replace('/', ''))            
            
    # remove strings containing some special characters ('|', '=', '+')
    words_no_special_chars = []
    for word in words_no_urls:
        if not ('|' in word or '+' in word or '=' in word):
            words_no_special_chars.append(word)   

    
    # remove string with size 1
    words_no_small = [word for word in words_no_special_chars if len(word)>1]
    
    return words_no_small

In [10]:
nltk.download('punkt')
nltk.download('stopwords')
preprocessed_corpus = [preprocess_document(doc) for doc in corpus]

# save and load
joblib.dump(preprocessed_corpus, 'preprocessed_corpus.pkl') 
preprocessed_corpus = joblib.load('preprocessed_corpus.pkl')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# convert corpus to a list of strings
preprocessed_corpus_string = list(map(lambda doc : ' '.join(doc) , preprocessed_corpus))

# save and load
joblib.dump(preprocessed_corpus_string, 'preprocessed_corpus_string.pkl')
preprocessed_corpus_string = joblib.load('preprocessed_corpus_string.pkl')

## Μετατροπή σε TFIDF

Το πρώτο βήμα θα είναι λοιπόν να μετατρέψετε το corpus σε αναπαράσταση tf-idf:

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create sparse tf_idf representation
vectorizer = TfidfVectorizer()
vectorizer.fit(preprocessed_corpus_string)
corpus_tf_idf_plain = vectorizer.transform(preprocessed_corpus_string)

Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης.

Προσοχή: ο TfidfVectorizer έχει κάποιες δυνατότητες προεπεξεργασίας παρόποιες με αυτές που αναφέραμε στην προηγούμενη ενότητα. Ό,τι προεπεξεργασία μπορείτε να κάνετε που χρειάζεται ως είσοδο μόνο το κάθε document ξεχωριστά, κάντε την στο πρώτο βήμα της προεπεξεργασίας. Αν χρειάζεται γνώση των συνολικών στατιστικών της συλλογής, κάντε την με τον TfidfVectorizer.

In [13]:
print(corpus_tf_idf_plain.shape)

(5000, 51662)


In [14]:
corpus[1300]

"marie skinner  is a gold digger with her hooks out for devoted middle-aged family man j.c. hudson , a portly real estate tycoon, who falls for her when she contrives to meet him. when his wife  and grown children, ruth  and billy  discover him dancing with marie at a nightclub, j.c. leaves home the next day. ruth seeks out marie to shoot her, but is interrupted by marie's boyfriend, jazz hound babe winsor , who takes a shine to her. when judson walks in on them he condemns her licentiousness, but is forced to face his double standard when he witnesses a violent argument between marie and babe. full of contrition, j.c. returns to home and hearth and the bosom of his loving family.jhailey@hotmail.com plot summarywollstein, hans j. plot synopsistcm synopsis full synopsis"

In [15]:
# dataframe version of the tf_idf corpus
df_tf_idf_plain = pd.DataFrame(corpus_tf_idf_plain.toarray(), columns = vectorizer.get_feature_names_out())

## Υλοποίηση του συστήματος συστάσεων

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση `content_recommender` με τρία ορίσματα: `target_movie`, `max_recommendations` και `corpus_type`. Στην `target_movie` περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, `max_recommendations` στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής: 
- για την ταινία-στόχο, θα υπολογίζετε την [ομοιότητα συνημιτόνου](https://en.wikipedia.org/wiki/Cosine_similarity) της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο `corpus_type`.
- με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (`ID`) των ταινιών. Παράδειγμα: αν η ταινία με index 1 έχει ομοιότητα συνημιτόνου με 3 ταινίες \[0.2 1 0.6\] (έχει ομοιότητα 1 με τον εαύτό της) ο ταξινομημένος αυτός πίνακας indices θα είναι \[1 2 0\].
- Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
- Για τις `max_recommendations` ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [16]:
import scipy as sp
from IPython.display import display, Markdown, Latex

# returns the row of a md table reffering to movie with index ind. It is called from content_recommender
def createRow(ind,similarity,extra='',bold=False):
    if bold:
        return "|**" + str(ind) + ' ' + extra + "**|**" + titles[ind][0]+ "**|**" + str(corpus[ind]) + "**|**" + str(categories[ind]) +\
            "**|**" + str(similarity) +"**|\n"
    
    return "|" + str(ind)+extra + "|" + titles[ind][0]+ "|" + str(corpus[ind]) + "|" + str(categories[ind]) +\
    "|" + str(similarity) +"|\n"

def content_recommender(target_movie, max_recommendations, corpus_type):
    (height, width) = corpus_type.shape
    distances = np.zeros(height)
    for i in range(height):
        distances[i] = 1 - sp.spatial.distance.cosine(corpus_type[target_movie].toarray()[0],
                                                  corpus_type[i].toarray()[0])
        
#     order = sorted(range(len(distances)), key = (lambda k : (distances[k], len(preprocessed_corpus[k]))), reverse=True)
    order = sorted(range(len(distances)), key = (lambda k : (distances[k], random.randint(0,5000))), reverse=True)
    order.remove(target_movie)

    disp = "|Movie|Title|description|categories|cosine similarity|\n|:-:|:-:|:-:|:-:|:-:|\n"
    disp += createRow(target_movie, 1.0, extra="(target)", bold=True)
    for i in range(0,max_recommendations):
        disp += createRow(order[i], distances[order[i]])
    display(Markdown(disp))
    print()
    print(disp)

In [17]:
# import random
# r = random.randint(1,5000)
# content_recommender(1580,3,corpus_tf_idf_plain)

## Βελτιστοποίηση του TfidfVectorizer

Αφού υλοποιήσετε τη συνάρτηση `content_recommender` χρησιμοποιήστε την για να βελτιστοποιήσετε την `TfidfVectorizer`. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό `max_recommendations` (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το `ID` τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την `TfidfVectorizer` για τα συγκεκριμένα `ID` ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό `max_recommendations`. Παράλληλα, όσο βελτιστοποιείτε την `TfidfVectorizer`, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών. 

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του `TfidfVectorizer` έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων. 




Ξεκινάμε σημειώνοντας κάποια IDs ταινιών που επιστρέφουν καλά αποτελέσματα με χρήση του _Out-of-the-Box_ TfidVectorizer, για `max_recommendations = 2` : 1580, 3784, 2359, 2259, 1448, 3786, 1046. Παρακάτω δείχνουμε και σχολιάζουμε τα αποτελέσματα για αυτά τα IDs.

In [18]:
# content_recommender(1046,2,corpus_tf_idf_plain)

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**1580 (target)**|**Morning Call**|**the rich and successful dr. manning is called out in the middle of the night to visit a private patient. he never returns and the next morning his wife annette  finds him missing. soon after, she receives a ransom note demanding £5,000 for his release. the police are alerted and soon annette is trying to deliver the money to various drop-off points specified by the kidnapper in telephone calls to her. the police keep watch, hoping to catch the kidnapper in the act of retrieving the money, but every attempts ends in failure as he fails to show up, realising the locations are being watched. annette hires a private detective nick logan  to make his own investigations. manning is found dead, and the police decide to use annette as bait to catch his killer. they publicise that she has heard his voice in the phone calls and will be able to identify it if she hears it again, hoping that the threat will flush him out in order to try to get her out of the way. logan now begins to work together with the police, and they finally succeed in cornering the killer, who reveals a surprising motive for his actions.**|**['"Crime Fiction",  "Thriller"']**|**1.0**|
|1376|Voice of a Murderer|one day, the only son of famous news anchor, han kyung-bae, disappeared without a trace. soon, the kidnapper calls the nine-year-old boy’s mother, oh jisun, demanding $100,000 as ransom. the police assign veteran detective kim wook-jung to the case and assemble a task force under his command. however, the kidnapper constantly outwits the police traps and keeps calling the parents with yet another instruction for the money drop. the only clue the police and the parents have is the kidnapper's voice recorded on tape. the parents become restless and angry as days turns into weeks, and one day, when the kidnapper calls han kyung-bae with another set of instructions, he turns the situation completely around and demands a direct confrontation.  sol kyung-gu kim nam-ju kim young-cheol kang dong-won song yeong-chang go soo-hee|['"Thriller",  "Drama",  "World cinema"']|0.17808466870924622|
|2877|Bolero|the film opens shortly before world war i. raoul de baere  is a dancer from new york city, aiming to become king of the european nightclub circuit. he tries to get annette  to be his dancing partner, but she refuses. he recruits helen hathaway  instead and devises a very athletic routine to be accompanied by ravel's boléro . he falls in love with helen, but this is unrequited, and she marries lord robert coray . raoul serves in the united states army in world war i and emerges with a weakened heart. nevertheless, he vows to carry on with his career. he opens a smart nightclub in paris, and recruits annette  as his new partner. on the opening night, as he is about to start the show, he finds her drunk and unable to perform. fortunately, helen is in the audience and agrees to stand in. raoul hopes that she will rejoin him. desperate to impress the audience, he overdoes his athletic routine, collapses and dies.|['"Musical",  "Drama",  "Black-and-white"']|0.14185063810733212|

**Σχολιασμός**: Εδώ η πρώτη μόνο σύσταση είναι καλή, καθώς η ταινία που συστήνεται είναι μια ιστορία απαγωγής όπως είναι και η ταινία στόχος.

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**3784 (target)**|**How to Console Widows**|** a bankrupt playboy makes a plan to obtain money. he reads in the news about the death of three wealthy married rich men who die in a plane crash. disguised as a ghost, that night he visits the three widows. scared, they not only give him money, but also consent to having sex with him. when the three women become pregnant, the father hires a priest to exorcise the little ghosts from them.**|**['"Comedy"']**|**1.0**|
|2831|Lonesome Ghosts|the ajax ghost exterminators are hired by telephone to drive out four ghosts from a haunted house that has long been abandoned. unbeknownst to them, they were hired by the ghosts themselves, who are bored because nobody has visited the house they had long been haunting . they wish to play tricks on the living, and do so through a series of inventive, annoying pranks. the exterminators arrive, enter and announce themselves, but there is nobody to receive them. mickey decides they should get to work anyway, and the three split up to hunt the ghosts individually. the exterminators are toyed with at every turn; mickey is driven upstairs and tries to open a door, which opens in a splash of water. donald, meanwhile, is whacked with a wooden board and is scared away by the sounds of banging chains and dishes. goofy, in a bedroom, becomes tangled in a dresser and stabs his own rear with a pin, mistaking his blue pants for a ghost. in the end, the three exterminators accidentally become covered in molasses and flour, making them look like ghosts and consequently scaring the actual ghosts out of the house in a panic. the ghost hunters stand victorious, having driven the spirits out of the house, although not exactly certain how. donald smugly assumes the ghosts fled in capitulation to their superior tactics. goofy offers what is considered the most memorable quote while warily looking around him: "i'm brave! but i'm careful." other quotes include donald's observation: "so you can't take it, you big sissies!", or another goofy quip: "i ain't scared of no ghosts!" .|['"Short Film",  "Family Film",  "Comedy",  "Animation"']|0.16210872962854395|
|2504|The Haunted Barn|john moon  is a businessman interested in ghosts, who visits an old shed hoping to meet one. he is visited by two lovers  seeking to elope, a mysterious captain sturdy , and three others. |['"Comedy film",  "Drama"']|0.11710326987818331|

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**2359 (target)**|**Graveyard Disturbance**|**five teen friends after stealing for a store end up in a gothic cemetery. while there they enter a tavern where the bartender challenges them to stay the night in a cursed crypt and if they survive the night, they get a treasure.**|**['"Thriller",  "Horror",  "World cinema",  "Supernatural",  "Gothic Film",  "Creature Film"']**|**1.0**|
|2718|The Iron Rose|a woman and man who met at a wedding reception decide to go on a date. he meets her at a railway station and they go for a picnic and a bike ride. they come to the outside of a cemetery and decide to go in. once inside the huge cemetery, the woman becomes frightened. he calms her and persuades her to enter a crypt with him. a strange man watches the couple. the man and woman make love in the crypt. a clown places some flowers on a nearby grave and leaves. an old woman closes the cemetery gates. when the couple finally exit the crypt, night has fallen and they cannot find their way to the exit. they begin to panic. they discover a small building; inside are several child-sized coffins with small skeletons inside. the woman becomes hysterical, and exhibits bizarre behaviour and radical personality changes. she locks her lover in the crypt and he suffocates. dawn finds the woman dancing around the cemetery, and later she re-enters the crypt. the old woman reopens the cemetery gates. finding the crypt open, she seals the door and puts flowers on top of it.|['"Horror",  "Psychological thriller",  "World cinema"']|0.2955775291515881|
|1423|Ganda Berunda|ganda berunda tells the story of two friends who search for a treasure which is secretly kept in remote island. the villain simultaneously appoints a group of men to locate and find the treasure and assures them of a share in the treasure.|['"Adventure"']|0.15769111413251435|

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**2259 (target)**|**Signs of Life**|**during world war ii, three german soldiers are withdrawn from combat when one of them, stroszek, is wounded. they are assigned to a small coastal community on the greek island of kos while stroszek recuperates. the men become increasingly stir crazy in their uneventful new assignment. stroszek eventually goes mad.**|**['"Drama",  "World cinema"']**|**1.0**|
|3791|The Crossing of the Rhine|the film tells the story of two french soldiers in the aftermath of the german invasion of france who become forced labourers on a german farm under the service du travail obligatoire programme , but become involved in the lives of their captors.|['"Drama",  "War film"']|0.07788506377658422|
|4193|Tokyo Park|a college student, aspiring to become a photographer, receives a job to follow a client's girlfriend. this assignment changes his relationship with women around him.|['"Japanese Movies"']|0.070694697368191|

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**1448 (target)**|**Donald's Snow Fight**|**the short begins with donald wiping the fog off his glass door, excited to see the snow. donald says that he is excited about snow. he opens his door and soon gets his bill frozen from the cold. he gets a fur overcoat and pulls a sled, while singing jingle bells. when donald reaches the top of the hill, he notices his nephews, huey, duey, and louie, at the bottom, building a snowman, while singing for he's a jolly good fellow. tempted, donald crashes his sled into their snowman and thus his nephews, resulting each of them to "wear" a part of their snowman. while donald is laughing himself silly, huey, duey, and louie plot revenge. the next scene shows donald's nephews piling snow and a duck-like head on a boulder. donald is scraping snow off of his foot when he hears his nephews yelling, "that's uncle donald! that's uncle donald! that's uncle donald!", while standing by the degrading snowman of donald. "oh, yeah?!" donald replies, and he shortly crashes himself into the snowman-disguised boulder. the shockwaves from the crash shake off all of the furs on his overcoat, save for one, that donald touches with his finger to stop its shaking and pulls out. now donald's nephews are laughing themselves silly and donald pulls off the remains of his overcoat. he dashes off to start chasing them. donald runs so fast, that he blows the snow off of a kissing couple. the nephews dash across a frozen pond to the center, where an imposing fortress made of ice and snow stands. donald bounces off the mattress door that they throw up, causing him to ricochet across the ice  and crashing into a tree. snow piles on top of him  until he is "transformed" into a rhinoceros in a cage. as a victory song, the trio sings a verse of london bridge while spinning and jumping in a circle. donald bursts out of the cage, squawking angrily, and declares war on his nephews. later on, donald yells at his nephews from a snow battleship, that he created, while his nephews simply mock him from their snow fortress. donald starts a small barrage of snowballs, which results the nephews into becoming like bowling pins, waiting for donald to strike them out. donald then pulls out a bucket of water, dips a snow chunk in it, and creates an ice ammunition  he launches it and it splits his nephews' flagpole in three spots, "spanking" all three of them. donald asks soon after if they surrender, and they clearly refuse. they start creating snow chunks of their own, while also getting three other objects: a picnic basket, some rope, and some mouse traps. they place the mouse traps in rocket-shaped components and stick them in the picnic basket now with the string tied to it. huey, dewey, and louie soon launch the basket and release the missiles, with donald getting hit by several of them. with huey commanding them with a wooden sword, louie and dewey started launching snowballs from a catapult. donald gets his hat spun off his head and yells, "that's unconstitutional!" before getting more snowballs launched at him. as his barrier starts shrinking with each hit, donald looks up, gets smacked in the face, and is launched him into a smaller part of the ship. being the little cheaters that they are, the nephews spear hot coal onto arrows  and launch them into the area where donald currently is, creating a skull-like structure. as a final attack, they launch a flaming snowball into a part of donald's ship. this leads donald to scream at them "i'll exterminate you!" this causes the ship to start melting and end up with donald frozen in a palm tree-shaped prison of ice. to celebrate their victory, the triplets perform a  ceremonious native american dance around the icicle; this latter performance was edited out of later broadcasts of the clip, due to perceived political incorrectness. this short was featured as part of the edited collection, donald and his duckling gang, from 1979, and is featured in from all of us to all of you which is shown on tv every christmas in the scandinavian countries. this short was one of the many featured in donald duck's 50th birthday**|**['"Short Film",  "Animation"']**|**1.0**|
|2154|The Autograph Hound|donald duck tries to enter a hollywood studio so he can search for celebrities willing to sign their autograph. a police officer guarding the gate prevents him from entering the building. donald manages to sneak inside by climbing on the limousine with greta garbo so that it seems he's riding along with her. the police officer discovers he's been fooled and chases donald, who enters a room with the name "mickey rooney" on it. inside, mickey rooney is dressing up in front of the mirror, when donald asks him for his autograph. rooney writes his name in donald's book and makes it disappear and reappear with a magic trick. donald, who is not amused, tries to impress rooney by doing a similar trick with an egg. the egg is however obviously hidden under donald's hat and rooney who is aware of this, crushes it, laughing loudly. donald gets extremely angry and starts waving his fists, while rooney manages to put a violin in donald's hands. when donald discovers he has been tricked for the third time he throws the violin at rooney. rooney ducks and the instrument lands in the face of the police officer. alarmed, donald runs away and hides under a bell-jar carried by actor henry armetta. when the police officer discovers donald's hiding place the duck runs to another film set full with ice. there he meets sonja henie and asks her for an autograph. henie signs her name by skating it in the ice, so that donald has to carry it with him. while walking in a desert setting donald discovers the ice has melted. he notices a tent with the silhouettes of three belly dancing arabic women, who turn out to be the ritz brothers. excited, he asks them for their autographs, but behaving like screwballs they jump on donald and sign their group name on his buttocks. an enraged donald throws a paint can at their heads, but it hits the face of the police officer instead. again donald has to flee and he runs to a castle with the sign the road to mandalay, which turns out to be just a model. after bumping his head into it and realizing his mistake he runs into another direction. on a pair of stairs he bumps into shirley temple. she, too, recognizes him and asks for an autograph. they both sit down to sign each other their autographs and donald, excited he has his first real autograph, jumps in the air with joy. then suddenly the police officer grabs him. shirley tells the police officer to leave him alone and he drops donald on the floor in surprise. "donald duck? did you say "donald duck?". other hollywood actors hear his comment and enthusiastically rush to donald to ask him to sign his autograph for them. (in chronological order: greta garbo, clark gable, the andrews sisters, charlie mccarthy, stepin fetchit, roland young, the lone ranger riding his horse silver, joe e. brown, martha raye, hugh herbert, irvin s. cobb, [[edward arnold . when the police officer asks donald to sign his autograph book and offers him his pen, donald squirts ink in the policeman's face. while the ink drips from the officer's face and writes donald's name on his chest, donald laughs hysterically.|['"Family Film",  "Comedy film",  "Animation"']|0.5590006688961914|
|807|Tea for Two Hundred|while camping in the mountains, donald duck sits down to eat a picnic. soon a line of ants emerges from a nearby bush and marches past donald, each carrying a grain harvested in the wild. donald takes interest in the straggler, a small ant who seems to be carrying a bean several times his body weight. donald begins to test the ant to see how much he can take. the ant, seemingly unaware of donald, continues to march. finally donald balances several food items from his picnic on the any, which the ant carries. but when donald makes the ant walk a tight rope, donald plucks the string causing the ant to fall into a large custard pie. the ant is angry with donald at first, but when he tastes the pie and sees all of donald's food, he becomes elated. he enthusiastically runs back to the ant colony and tells the ants about all of donald's food. the ants call the colony together using mushrooms as drums. later, while donald is sleeping against a tree next to his picnic basket, a small team of ants sneak up on him. the open the basket, but donald, still asleep, instinctively puts his hand over the basket to keep the ants out. then all of the ants fill in around donald and pick him up. they carry him over to a cliff where they throw him off into the water. as an angry donald races back up the cliff, the ants are carrying food items out of his basket into their hole. in a mock game of football donald is steamrolled by the ants. as he tries grab his food directly from their hole, the ants pull of donald's shirt as well. he then puts on a barrel to protect his modest, and shoves several sticks of dynamite down the anthill and ignites it. immediately after the explosion, donald laughs that he has seemingly had the last say against the ants. yet moments later, the ground around donald starts to crack. donald is standing near the cliff, and the explosion of the dynamite causes a large part of the cliff to fall away, and donald falls into the water a second time, screaming as he does. in the final sense, the ant colony congregates around a large cupcake. the little ant whom donald had been harassing runs to the top and takes a big bite out of the cherry.|['"Short Film"']|0.5429302028869584|


|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**3786 (target)**|**The Elusive Pimpernel**|**during the french revolution, the scarlet pimpernel, , who is really sir percy blakeney in disguise, risks his life to rescue french noblemen from the guillotine and take them across the english channel to safety. as cover, sir percy poses as a fop at court, and curries favor with the prince of wales  by providing advice about fashion, but secretly he leads the league, a group of noblemen with similar views. chauvelin, french ambassador of the revolution to england  wants to find out who the pimpernel is and bring him in to meet his fate under french justice. when evidence points to sir percy, chauvelin blackmails blakeney's wife, marguerite  by threatening to expose her criminal brother armand , but marguerite doesn't believe her husband is capable of being the daring pimpernel.**|**['"Costume drama",  "Adventure"']**|**1.0**|
|3918|The Elusive Pimpernel|a foppish english aristocrat secretly rescues people from the guillotine during the french revolution.|['"Silent film",  "Drama",  "Adventure"']|0.2155169264082194|
|1307|Stony Island|richie bloom  is the last white kid on the block in the tough stony island neighborhood on chicago’s south side. a sweetly naive 18 year-old guitar player, he dreams of putting together his first band. richie has the perfect musical collaborator in his best friend, kevin tucker , who has a voice second only to james brown. richie buys a used les paul model electric guitar from jerry domino , who owns a tropical fish emporium, but also runs a few other "businesses" on the side. percy price  is a local legend on the saxophone. the kids used to listen to him practicing at night when they were growing up. percy agrees to help richie and kevin to form a group, letting them practice at the local mortuary where he works. however, the owner lewis moss  wants to sell the business to the local alderman, who plans to turn it into a disco. richie, kevin and percy begin to put the band together, searching the city for horn players. richie stumbles upon harold tate , a hillbilly sax player, while harold is washing windows at the music store where richie works. harold's mean uncle yanks him away from an impromptu jam session before richie can get his name or address. richie scours the city to find the young wunderkind and then hilariously conspires with kevin to get harold a horn to play. the band starts coming together, and percy even manages to set up their first gig in a club where blues legend b.b. king often performs. however, annoyed with the music during a customer’s funeral, alderman waller orders mr. moss to fire percy, and the group must find a new place to practice. fortunately, jerry domino lets the kids use a rehearsal space above his tropical fish emporium. during a rehearsal, percy has a heart attack while playing a solo, and dies. stunned by their mentor’s death, the kids sadly take his body back to the mortuary for a courtesy funeral. however, mr. moss informs them that percy's insurance policy had lapsed, and callously demands they pay the funeral cost or take percy's body to the county morgue. but mr. moss makes the mistake of calling ronnie roosevelt , a man listed as "next-of-kin" on percy's lapsed insurance policy. mr. moss tricks ronnie into believing that he is the beneficiary of the policy, when in fact, mr. moss only hopes to sell him an expensive funeral service for percy. ronnie is a new orleans piano man with a soulful voice and a voodoo attitude. he is outraged to learn that he was duped into coming to chicago and he threatens mr. moss with an especially unpleasant form of death. as if by magic, mr. moss is found dead at his expensive home, and the kids decide to take matters into their own hands. when kevin learns it will be a closed casket ceremony for mr. moss, he and kevin switch percy's and mr. moss' bodies, so that percy will get the expensive funeral. that night the mortuary is filled with music and musicians as they celebrate a new orleans style wake. mr. moss' body ends up in the autopsy room of the county morgue. the stony island band’s debut is a smash. the evening is dedicated to percy, whose spirit arrives just in time to finish the solo he was playing when he died.|['"Drama"']|0.18893208109039938|


|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**1046 (target)**|**God's Sentence**|**antonio, desperate man who cannot maintain a job to support his mother, dona lídia, and his sister marta, after his father, a banker and patriarchal head of the family, is killed. later, he boards a group of criminals who plan a robbery and they want and him to be the driver. the production of the film was interrupted after some problems, including the death of actress conchita espanhol.**|**['"Action"']**|**1.0**|
|112|Mafioso|antonio badalamenti, a sicilian who has been settled for many years in northern italy and is employed in a car factory in milan, takes a vacation with his family, leaving behind the modern conveniences of his home in northern italy, to visit his ancestral home in sicily and introduce his blond, northern italian wife, marta, to his mother, father and other relatives back home. while his wife suffers in the comparatively rustic conditions of her husband's hometown and has trouble adapting to the culture of sicily, antonio becomes reacquainted with his childhood friends. he also pays a visit to the local don, don vincenzo, who is a crime boss. the don smooths over some problems antonio had with a deal to buy some property on the island, and in return, antonio is tasked with carrying out a hit for the mob. as an outsider with no strings attached and a crack shot, antonio is seen as a perfect candidate. while his wife is sleeping one night, antonio leaves for what is purportedly a hunting trip with his friends. in reality, he is put inside a wooden crate and smuggled aboard an airplane into the united states, where he goes to new york city to carry out his task. the job done, he is returned to sicily in the same manner and arrives back at home as if he's just finished the hunting trip.|['"Crime Fiction",  "World cinema",  "Drama",  "Comedy",  "Black-and-white"']|0.16645866168750534|
|345|The Terror|a group of criminals carry out a daring robbery of an armoured van. two of the criminals are betrayed by the mastermind of the operation. after ten years in prison they come out and search for the man behind the crimes who betrayed them. but the police are on their tail also wanting to find out who was behind the robbery.  |['"Thriller",  "Crime Thriller",  "Suspense"']|0.1429437537555408|

In [19]:
# create sparse tf_idf representation
vectorizer_improved = TfidfVectorizer(max_df=1.0, min_df=0.005)
vectorizer_improved.fit(preprocessed_corpus_string)
corpus_tf_idf_improved = vectorizer_improved.transform(preprocessed_corpus_string)

In [20]:
print(corpus_tf_idf_improved.shape)

(5000, 4073)


In [21]:
# dataframe version of the tf_idf corpus
df_tf_idf_improved = pd.DataFrame(corpus_tf_idf_improved.toarray(), columns = vectorizer_improved.get_feature_names_out())

In [22]:
r = random.randint(1,5000)
content_recommender(r,3,corpus_tf_idf_improved)

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**3217 (target)**|**Kadhal Enum Nadhiyinile**|**in kadhal enum nadhiyinile, shankar plays an arrogant brat of college. he loves seetha, his colleague, but seetha loves pandian, her classmate. the film deals with the mental agony of seetha to choose between her lovers. the film has an unexpected twist and has a tragic end.**|**['"Romance Film",  "Drama"']**|**1.0**|
|3758|Japon İşi|veysel loves a singer named başak. but she don't loves him. one day veysel saves the life of a japanese tourist. tourist sends to veysel a gift from japan. the gift is a robot like singer başak.|['"Romance Film",  "Science Fiction",  "Comedy film"']|0.1589539439995009|
|3901|Dubai|raffy and andrew were orphaned as kids and had only each other to depend on. raffy has spent the last nine years of his life working in dubai. his ultimate goal is to fulfill a lifelong dream: to eventually move to canada with his younger brother, andrew. the alvarez brothers are finally united when andrew goes to dubai. in dubai, andrew meets faye, one of raffy's many girlfriends. they hit it off well in spite of their age difference. she becomes his guide, comfort and lover. seeing the two together, raffy realizes that he still and really loves faye. when andrew discovers that raffy still loves faye, conflict arises between the brothers, almost severing the ties that bind them. in the end, what they choose and achieve are not as planned, but their experiences in dubai lead to new beginning in their lives.|['"Romance Film",  "Drama",  "Filipino Movies"']|0.15890319550536347|
|3265|Dobara|shroff plays ranbir saigal and chaudry plays anjali, a happily married couple who find themselves in trouble when visited by an escapee from a mental asylum, ria deshmukh . soundtrack- the music of dobara, was composed by anu malik and all lyrics were penned by javed akhter. it is one of the melodious music of 2004. tracklisting- 1. mujhse kyun - alka yagnik 2. gunja hua - alka yagnik 3. humnasheen- alisha cinoy, anu malik 4. pyaar mere nu - jasbeer jassi 5. tum abhi  - hariharan 6. tum abhi  - alka yagnik|['"Romantic drama",  "Romance Film",  "Drama",  "World cinema"']|0.1566223923235135|



|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**3217 (target)**|**Kadhal Enum Nadhiyinile**|**in kadhal enum nadhiyinile, shankar plays an arrogant brat of college. he loves seetha, his colleague, but seetha loves pandian, her classmate. the film deals with the mental agony of seetha to choose between her lovers. the film has an unexpected twist and has a tragic end.**|**['"Romance Film",  "Drama"']**|**1.0**|
|3758|Japon İşi|veysel loves a singer named başak. but she don't loves him. one day veysel saves the life of a japanese tourist. tourist sends to veysel a gift from japan. the gift is a robot like singer başak.|['"Romance Film",  "Science Fiction",  "Comedy film"']|0.1589539439995009|
|3901|Dubai|raffy and andrew were orphaned as kids and had only each other to depend on. raffy has spent the last nine years of his life working in dubai. his ultimate goal is to fulfill a lifelong dream: to eventually move to canada with his younger brother, andrew.

## Βαθιά μάθηση: δημιουργία corpora με χρήση word emmbeddings

Η προσέγγιση της κατασκευής μόνο μέσω tfidf του συστήματος συστάσεων έχει διάφορα μειονεκτήματα. Θα μας ενδιέφερε λοιπόν να δούμε αν μπορούμε να χρησιμοποιήσουμε για τις λέξεις **εμφυτεύματα (embeddings)**, δηλαδή τις πυκνές διανυσματικές αναπαραστάσεις για τις λέξεις που μας δίνει το μοντέλο **Word2Vec**

Ωστόσο, το dataset της κάθε ομάδας είναι πολύ μικρό για να εξάγουμε τα δικά μας word embeddings (και να είναι καλά). Για το λόγο αυτό θα χρησιμοποιήσουμε τη μεθοδολογία της Βαθιάς Μάθησης που είναι η **Μεταφορά Μάθησης (Transfer Learning).**.

Στη μεταφορά μάθησης ουσιαστικά μεταφέρουμε τη γνώση που έχει αποκτήσει ένα ήδη εκπαιδευμένο (και κατά κανόνα πολύ μεγάλο) σύστημα. Η μεταφορά γίνεται διαμέσου των τιμών των βαρών που έχει προσδιορίσει μετά το πέρας της εκπαίδευσης.

Στην περίπτωσή μας, δεν μας ενδιαφέρουν τόσο τα ίδια τα βάρη των μοντέλων από τα οποία θα κάνουμε μεταφορά μάθησης. Κάτι τέτοιο θα μας ενδιέφερε αν π.χ. θέλαμε να συνεχίσουμε την εκπαίδευση στα δικά μας κείμενα. Μας ενδιαφέρουν όμως τα ίδια τα εμφυτεύματα, δηλαδή τα embeddings (διανύσματα διαστάσεων $m$) που έχει μάθει το νευρωνικό για το λεξιλόγιο του (vocabulary). To vocabulary σε τέτοια μεγάλα νευρωνικά θα είναι πιθανότατα υπερσύνολο του δικού μας.

### Μεταφορά μάθησης εμφυτευμάτων



#### Εμφυτεύματα του Gensim-data
Το Gensim περιλαμβάνει αρκετά προεκπαιδευμένα μοντέλα εμφυτευμάτων Word2Vec. Με το επόμενο κελί παίρνουμε τη λίστα τους.

In [23]:
# !pip install -U gensim
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


Τα μοντέλα αυτά βρίσκονται στο [αποθετήριο Gensim-data](https://github.com/RaRe-Technologies/gensim-data) όπου μπορείτε να βρείτε και την τεκμηρίωσή τους. Η φόρτωση των μοντέλων αυτών γίνεται με τη συνάρτηση `gensim.downloader.load`.

#### Άλλα εμφυτεύμαατα
Μπορείτε να βρείτε προεκπαιδευμένα εμφυτεύματα και από πηγές εκτός του Gensim. Για παράδειγμα:

- [Google News dataset](https://code.google.com/archive/p/word2vec/). Πρόκειται για προ-εκπαιδευμένα διανύσματα που έχουν εκπαιδευτεί σε μέρος του συνόλου δεδομένων Google News (περίπου 100 δισεκατομμύρια λέξεις). Το μοντέλο περιέχει διανύσματα 300 διαστάσεων για 3 εκατομμύρια λέξεις και φράσεις.
- [Amazon BlazingText](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). Το BlazingText δεν είναι μόνο προεκπαιδευμένα εμφυτεύματα αλλα και βελτιστοποιημένες υλοποιήσεις των αλγορίθμων Word2vec για την επεξεργασία κειμένου. Προυπόθεση είναι να δουλέψει κανείς στο SageMaker.

Οι διαδικασίες φόρτωσης embeddings από εξωτερικά δεδομένα μπορεί να είναι ελαφρά διαφορετικές από αυτή του Gensim.



#### Παρατηρήσεις

*   Επαναλαμβάνουμε ότι στην εργασία αυτή δεν μας ενδιαφέρουν τα ίδια τα μοντέλα αλλά το να μπορούμε για μία λέξη του λεξιλογίου μας να μπορούμε να βρούμε το embedding (διάνυσμα) που της αντιστοιχεί στο εκάστοτε προεκπαιδευμένο μοντέλο. 

*   Επίσης, δεν θα χρησιμοποιήσουμε την `Phrases` για να βρούμε bigrams στο dataset μας όπως θα ήταν το ορθότερο, καθώς αυτό θα απαιτούσε την συνέχιση της εκπαίδευσης του μοντέλου σε νέο λεξιλόγιο με πολύ λίγα νέα δεδομένα.


 ### Δημιουργία corpora βασισμένων στα εμφυτεύματα

Για να μπορέσουμε να ενσωματώσουμε τη γνώση που υπάρχει στα προεκπαιδευμένα εμφυτεύματα στο δικό μας corpus θα προχωρήσουμε όπως περιγράφεται ακολούθως.

Για κάθε περιγραφή ταινίας $d$, η οποία αποτελείται από τις $N_d$ λέξεις $w_i$, το  $tfidf$ της κάθε λέξης $w_i$ δίνεται από τη σχέση:

$$ tfidf(w_i) = tf(w_i,d) \cdot idf(w_i)$$

Ταυτόχρονα, σε κάθε λέξη $w_i$ αντιστοιχεί ένα διάνυσμα $W2V(w_i)$ από το μοντέλο εμφυτευμάτων που έχουμε εισάγει. Τα διανύσματα εμφυτευμάτων $W2V$ θα έχουν διάσταση $m$, ανάλογα το μοντέλο. 

Για κάθε ταινία d, μπορούμε να δημιουργήσουμε μια διανυσματική αναπαράσταση $W2V(d)$ διαστάσεων $m$ χρησιμοποιώντας το $tfidf(w_i)$ ως συντελεστή βαρύτητας για κάθε εμφύτευμα $W2V(w_i)$:

$$ W2V(d) = \frac{tfidf(w_1)\cdot W2V(w_i) + tfidf(w_2)\cdot W2V(w_2) + \dotsc  + tfidf(w_{N_{d}})\cdot W2V(w_{N_{d}})}{tfidf(w_1)+tfidf(w_2)+ \dotsc + tfidf(w_{N_{d}})}$$


#### build_tfw2v

Υλοποιήστε μια συνάρτηση `build_tfw2v` με ορίσματα:
- `corpus` που θα είναι το προεπεξεργασμένο dataset σας,
- `vectors` που θα είναι το μοντέλο που θα σας δίνει τα διανύσματα των εμφυτεύσεων vectors, και 
- `embeddings_size` που θα είναι η διάσταση των εμφυτευμάτων $m$.

H συνάρτηση αυτή θα επιστρέφει ένα νέο corpus που θα είναι ένας πίνακας 5000 (όσες οι ταινίες σας) x $m$ (το η διάσταση των εμφυτευμάτων). Ανάλογα ποιο μοντέλο χρησιμποιείτε για transfer learning ο πίνακας αυτός θα είναι διαφορετικός.

Μπορείτε πλεόν να καλείτε την `content_recommender` με διαφορετικά corpora στο όρισμα `corpus_type`. Σημειώστε ότι στο TFidfVectorizer χρησιμοποιουμε τη σειριακή μορφή των numpy arrays και ίσως σας χρησιμεύσει η `sparse.csr_matrix()` από την Scipy.

In [24]:
model_google_news = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [25]:
def build_tfw2v(corpus, vectors, embeddings_size):  
    # initialize 2d array with the correct shape
    arr_shape = (5000, embeddings_size)
    tfw2v = np.zeros(arr_shape)
    
    # compute the w2v representation for each movie
    row = 0
    for movie_description in corpus:
        
        # initialize numerator and denominator of the w2v expression
        numerator = np.zeros(embeddings_size)
        denominator = 0
        
        # compute w2v of the movie considering every word in the movie description
        for word in movie_description:
            # if an embedding exists for the word do the computation normally
            if word in vectors:
                numerator += df_tf_idf_plain[word][row] * vectors[word]
                denominator += df_tf_idf_plain[word][row]
            # else do nothing

            
        # final computation 
        tfw2v[row] = numerator / denominator
        row += 1
    
    return tfw2v
            
        

In [26]:
# create representation using google news 
tfw2v_google_news = build_tfw2v(preprocessed_corpus, model_google_news, 300)

# save and load
joblib.dump(tfw2v_google_news, 'tfw2v_google_news.pkl') 
tfw2v_google_news = joblib.load('tfw2v_google_news.pkl')

In [27]:
import random
from scipy import sparse
r = random.randint(1,5000)
content_recommender(r,3,sparse.csr_matrix(tfw2v_google_news))

|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**292 (target)**|**Bad Karma**|**a female mental patient  believing she is the reincarnated soul mate of jack the ripper, terrorizes her psychiatrist , whom she believes is her reincarnated lover. determined to find her man, and willing to use her sensuality to get to him, she breaks out of the mental institution and is determined to, at any cost, free him of his wife  and young daughter to restart their previous work as mass murderers, and lovers.**|**['"Thriller",  "Horror",  "Slasher",  "Psychological thriller"']**|**1.0**|
|1931|Opapatika|a narrator explains that according to buddhist belief the evolution of birth can be divided into three forms: *sangsethaca: to be born amongst rot and decay, as a worm or maggot. *anthaca: to be born from an egg, in the various forms of birds. *chalaphucha: to be conceived in the womb as a human or other mammal. then there is a fourth form, opapatika, which goes against buddhist beliefs because it involves suicide: a class of supernatural beings who are born out of suicide. however, the powers that they gain also have a negative effect. the narrator, it is revealed, is thuwachit , the loyal mortal henchman for the elderly sadok , an opapatika who is quickly decaying and needs to feast on the flesh of other opapatikas in order to sustain himself. he sends thuwachit out to capture others. four opapatikas remain: *paisol , a ruthless assassin who must bear all the scars and wounds of his victims. *jiras , an immortal opapatika, and the most powerful of them all. he considers his immortality a curse. *aruth , an invincible fighter by night, but weak in the daylight. *ramil , an adrenaline-fueled daredevil who can project a monstrous, ghostly creature to do his bidding. investigating the opapatikas is a private detective, techit , whom sadok transforms into an opapatika. techit has the psychic powers of a mind reader, but his power costs him the use of his five senses. techit is teamed up with thuwachit, who leads a vast paramilitary army against the four opapatika. the four immortals are also mysteriously drawn to a woman, pran .|['"Fantasy",  "Action",  "Horror"']|0.7872295508040598|
|670|Cure|yakusho plays kenichi takabe, an emotionlly repressed police detective with a mentally unstable wife. takabe investigates a series of bizarre murders. though each victim is killed in the same way, with a large "x" carved into their necks, the perpetrator is different each time. in every case the murderers are caught close to the scene of the crime, and although they readily confess to committing the crimes, they never have substantial motive and cannot explain what drove them to kill. takabe, together with a psychologist named sakuma, eventually determines that one man is the common thread among the murders, as each person he comes in contact with commits a killng shortly thereafter. the man, called mamiya, appears to have extreme short-term memory loss; he seems constantly confused about what day it is, where he is, and what his name is. he claims to recall nothing of his past. after takabe catches mamiya, he searches the man's apartment and discovers that mamiya used to be a student of psychology who studied mesmerism and hypnosis. takabe comes to realize that mamiya has no memory problems, and that he is instead a master of hypnosis, capable of planting criminal suggestions in strangers' minds by exposing them to repetitive sounds, the motion of water, or the flame of a lighter. mamiya is jailed and charged with incitement to murder. mamiya also finds takabe fascinating, possibly because he cannot force takabe to kill. takeba does have visions of his wife dead, however, and the more he researches mamiya, the more takeba feels that he might be losing his mind. his wife's strange behavior and concerns about his own mental stability lead him to have her committed to a mental hospital. when mamiya escapes, killing a policeman and a doctor in the process, takabe tracks him to a deserted building in the wilderness and shoots him. the film's final scenes suggest that takabe himself has become the master hypnotist, and that he is carrying on mamiya's bizarre work.|['"Crime Fiction",  "Thriller",  "Horror",  "World cinema",  "Psychological thriller",  "Japanese Movies",  "Crime Thriller"']|0.7742037427551066|
|2619|Macabre|a new orleans wife and mother carries on a torrid affair behind her family's back. but when a violent accident leaves her lover dead, the woman returns from a mental institution determined to pursue her forbidden desires. has she found a ghastly new way to satisfy her lust, or is sexual depravity all in the head?|['"Horror"']|0.7565461532624055|



|Movie|Title|description|categories|cosine similarity|
|:-:|:-:|:-:|:-:|:-:|
|**292 (target)**|**Bad Karma**|**a female mental patient  believing she is the reincarnated soul mate of jack the ripper, terrorizes her psychiatrist , whom she believes is her reincarnated lover. determined to find her man, and willing to use her sensuality to get to him, she breaks out of the mental institution and is determined to, at any cost, free him of his wife  and young daughter to restart their previous work as mass murderers, and lovers.**|**['"Thriller",  "Horror",  "Slasher",  "Psychological thriller"']**|**1.0**|
|1931|Opapatika|a narrator explains that according to buddhist belief the evolution of birth can be divided into three forms: *sangsethaca: to be born amongst rot and decay, as a worm or maggot. *anthaca: to be born from an egg, in the various forms of birds. *chalaphucha: to be conceived in the womb as a human or other mammal. then there is a fourth form, opapatika, which goes against

## Ανάλυση αποτελεσμάτων

### Σύστημα συστάσεων βασισμένο μόνο στο tfidf

- Σε markdown περιγράψτε τι προεπεξεργασία κάνετε στα κείμενα και γιατί.

- Περιγράψτε πως προχωρήσατε στις επιλογές σας για τη βελτιστοποίηση της `TfidfVectorizer`. 

- [Cherry-picking:](https://www.wikiwand.com/en/Cherry_picking) Δώσετε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν καλά αποτελέσματα μέχρι `max_recommendations` (τουλάχιστον 5) και σχολιάστε.

- [Nit-picking:](https://en.wikipedia.org/wiki/Nitpicking) Δώστε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν κακά αποτελέσματα και σχολιάστε.

- Ποια είναι συνολικά τα πλεονεκτήματα και μειονεκτήματα ενός recommender βασισμένου στο tfidf;

**Περιγραφή της προεπεξεργασίας**

* Αρχικά μετατρέπουμε όλους τους χαρακτήρες σε πεζούς.

* Στη συνέχεια αφαιρούμε stopwords. Αφενός αφαιρούμε κλασικά stopwords της Αγγλικής Γλώσσας. Αφετέρου επιλέγουμε να αφαιρέσουμε κάποιες λέξεις όπως '_film_' και '_movie_' που δεν περιμένουμε να προσθέτουν επιπλέον ουσία στην περιγραφή μιας ταινίας.

* Αφαιρούμε σημεία στίξης. Αρχικά αφαιρούμε όσα σημεία στίξης έχουν απομείνει ως μεμονωμένα strings από το tokenization. Στην συνέχεια αφαιρούμε τελείες που έχουν παραμείνει ως κομμάτια άλλων strings (λέξεων) παρά το tokenization. Το τελευταίο γίνεται για να αποφύγουμε να υπάρχουν δύο συμβολοσειρές στο corpus που είναι στην πραγματικότητα ίδιες, αλλά αναγνωρίζονται από το σύστημα ως διαφορετικές (π.χ. "hello" και "hello. ") 

* Επίσης ελέγχουμε για κάποιους ειδικούς χαρακτήρες (["'", "\`", "\"", "-", "_"]). Συγκεκριμένα, αφαιρούμε τελειώς απο το corpus strings που περιέχουν μόνο τέτοιους χαρακτήρες. Αν κάποιο string έχει μόνο ένα υποσύνολο χαρακτήρων που ανήκουν σε αυτούς τους ειδικούς χαρακτήρες, αφαιρούμε μόνο αυτούς τους ειδικούς χαρακτήρες (π.χ. το "-`hello" θα αντικατασταθεί από το "hello").

* Κατόπιν αφαιρούμε συμβολοσειρές που αναγνωρίζουμε ως urls, γιατί θεωρούμε ότι ένα url δεν θα έχει σημασιολογική αξία για την περιγραφή της ταινίας. Ως urls θεωρούμε συμβολοσειρές που περιλαμβάνουν τον χαρακτήρα '/'.

* Τέλος αφαιρούμε συμβολοσειρές μήκους 1 γιατί έχουν κατα πάσα πιθανότητα προκύψει στο tokenization και σε κάθε περίπτωση δεν περιμένουμε να έχουν ιδιαίτερα οφέλιμο περιεχόμενο.

**Βελτιστοποίηση του `TfidVectorizer`**

**Cherry-Picking: Παραδείγματα IDs με καλά αποτελέσματα**

**Nit-Picking: Παραδείγματα IDs με κακά αποτελέσματα**

### Σύγκριση και σχολιασμός με recommenders βασισμένων στο Word2Vec

- Υλoποιήστε recommenders που βασίζονται σε μεταφορά μάθησης και εμφυτεύματα. Χρησιμοποιήστε παραδείγματα για να υποδείξετε δυνατά και αδύναμα σημεία τους.

- Μπορείτε να σχολιάσετε τα recommenders που βασίζονται στο Word2Vec σε σχέση με το απλό μοντέλο tfidf, εξετάζοντας τις συστάσεις για ίδια ID.

- Μπορείτε επίσης να εξετάσετε συγκριτικά τα Word2Vec recommenders μεταξύ τους και πάλι βασιζόμενοι σε παραδείγματα.

- Οι παρατηρήσεις σας θα βασίζονται στην ανάλυση των ποιοτικών χαρακτηριστικών που είναι η σειρά και το σύνολο των συστάσεων. Ωστόσο, μπορείτε να συμπεριλάβετε και ποσοτικά χαρακτηριστικά όπως τους χρονους loading και συγκρότησης του corpus αλλά και της διαστατικότητας $m$.

Χρησιμοποιήστε όποια μορφή reporting κρίνετε καταλληλότερη: κείμενο, πίνακες, διαγράμματα.


## Πρακτικό tip - persistence αντικειμένων με joblib.dump

Καθώς στην δεύτερη εργασία καλείστε να δημιουργήσετε διάφορα corpora των οποίων η δημιουργία παίρνει χρόνο, υπάρχει ένας εύκολος τρόπος να αποθηκεύουμε μεταβλητές σε dump files και να τις διαβάζουμε απευθείας.

H βιβλιοθήκη [joblib](https://pypi.python.org/pypi/joblib) της Python δίνει κάποιες εξαιρετικά χρήσιμες ιδιότητες στην ανάπτυξη κώδικα: pipelining, παραλληλισμό, caching και variable persistence. Τις τρεις πρώτες ιδιότητες τις είδαμε στην πρώτη άσκηση. Στην παρούσα άσκηση θα μας φανεί χρήσιμη η τέταρτη, το persistence των αντικειμένων. Συγκεκριμένα μπορούμε με:

```python
joblib.dump(my_object, 'my_object.pkl') 
```

να αποθηκεύσουμε οποιοδήποτε αντικείμενο-μεταβλητή (εδώ το `my_object`) απευθείας πάνω στο filesystem ως αρχείο, το οποίο στη συνέχεια μπορούμε να ανακαλέσουμε ως εξής:

```python
my_object = joblib.load('my_object.pkl')
```

Μπορούμε έτσι να ανακαλέσουμε μεταβλητές ακόμα και αφού κλείσουμε και ξανανοίξουμε το notebook, χωρίς να χρειαστεί να ακολουθήσουμε ξανά όλα τα βήματα ένα - ένα για την παραγωγή τους, κάτι ιδιαίτερα χρήσιμο αν αυτή η διαδικασία είναι χρονοβόρα.

Ας αποθηκεύσουμε το `corpus_tf_idf` και στη συνέχεια ας το ανακαλέσουμε.

In [28]:
# import joblib

# joblib.dump(corpus_tf_idf, 'corpus_tf_idf.pkl') 



Μπορείτε με ένα απλό `!ls` να δείτε ότι το αρχείο `corpus_tf_idf.pkl` υπάρχει στο filesystem σας (== persistence):

In [29]:
# !ls -lh

και μπορούμε να τα διαβάσουμε με `joblib.load`

In [30]:
# corpus_tf_idf = joblib.load('corpus_tf_idf.pkl')

# Εφαρμογή 2.  Τοπολογική και σημασιολογική απεικόνιση της ταινιών με χρήση SOM
<img src="https://i.imgur.com/Z4FdurD.jpg" width="60%">

## Δημιουργία dataset
Στη δεύτερη εφαρμογή θα βασιστούμε στις τοπολογικές ιδιότητες των Self Organizing Maps (SOM) για να φτιάξουμε ενά χάρτη (grid) δύο διαστάσεων όπου θα απεικονίζονται όλες οι ταινίες της συλλογής της ομάδας με τρόπο χωρικά συνεκτικό ως προς το περιεχόμενο και κυρίως το είδος τους (ο παραπάνω χάρτης είναι ενδεικτικός, δεν αντιστοιχεί στο dataset μας). 

Διαλέξτε για την αναπαράσταση των documents αυτήν που πιστεύετε απέδωσε καλύτερα στο πρώτα σκέλος της άσκησης. Έστω ότι αυτή είναι η `my_best_corpus`.

Η έτοιμη συνάρτηση `build_final_set` θα ενώσει την αναπαράσταση που θα της δώσετε ως όρισμα `mycorpus` με τις binarized κατηγορίες `catbins` των ταινιών ως επιπλέον κολόνες (χαρακτηριστικά). Συνεπώς, κάθε ταινία αναπαρίσταται στο Vector Space Model από τα χαρακτηριστικά της αναπαράστασης `mycorpus` και τις κατηγορίες της.

Τέλος, η συνάρτηση δέχεται ένα ορισμα για το πόσες ταινίες να επιστρέψει, με default τιμή όλες τις ταινίες (5000). Αυτό είναι χρήσιμο για να μπορείτε αν θέλετε να φτιάχνετε μικρότερα σύνολα δεδομένων ώστε να εκπαιδεύεται ταχύτερα το SOM. 

Θα τρέχουμε τη συνάρτηση με `final_set = build_final_set(my_best_corpus)`.

In [31]:
def build_final_set(mycorpus, doc_limit = 5000, tf_idf_only=False):
    # convert sparse tf_idf to dense tf_idf representation
    dense_tf_idf = mycorpus.toarray()[0:doc_limit,:]
    if tf_idf_only:
        # use only tf_idf
        final_set = dense_tf_idf
    else:
        # append the binary categories features horizontaly to the (dense) tf_idf features
        final_set = np.hstack((dense_tf_idf, catbins[0:doc_limit,:]))
    # η somoclu θέλει δεδομ΄ένα σε float32
    return np.array(final_set, dtype=np.float32)

Στο επόμενο κελί, τυπώνουμε τις διαστάσεις του τελικού dataset μας. **Χωρίς βελτιστοποίηση του TFIDF** θα έχουμε περίπου 50.000 χαρακτηριστικά και ο θα είναι ανέφικτο να προχωρήσουμε στην εκπαίδευση του SOM.

In [32]:
# final_set.shape

## Εκπαίδευση χάρτη SOM

Θα δουλέψουμε με τη βιβλιοθήκη SOM ["Somoclu"](http://somoclu.readthedocs.io/en/stable/index.html). Εισάγουμε τις somoclu και matplotlib και λέμε στη matplotlib να τυπώνει εντός του notebook (κι όχι σε pop up window).

In [33]:
# install somoclu
!pip install --upgrade somoclu
# import sompoclu, matplotlib
import somoclu
import matplotlib
# we will plot inside the notebook and not in separate window
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for somoclu: filename=somoclu-1.7.5.1-cp37-cp37m-linux_x86_64.whl size=288023 sha256=c6450579b9c65c38b4163a4946cb6d2e65be808c61e13981c0d111a1aff118b0
  Stored in directory: /root/.cache/pip/wheels/d5/81/eb/22b023e9cac575c4b87d3f9a7318094124e5d8cf90600726fa
Successfully built somoclu


Καταρχάς διαβάστε το [function reference](http://somoclu.readthedocs.io/en/stable/reference.html) του somoclu. Θα δoυλέψουμε με χάρτη τύπου planar, παραλληλόγραμμου σχήματος νευρώνων με τυχαία αρχικοποίηση (όλα αυτά είναι default). Μπορείτε να δοκιμάσετε διάφορα μεγέθη χάρτη ωστόσο όσο ο αριθμός των νευρώνων μεγαλώνει, μεγαλώνει και ο χρόνος εκπαίδευσης. Για το training δεν χρειάζεται να ξεπεράσετε τα 100 epochs. Σε γενικές γραμμές μπορούμε να βασιστούμε στις default παραμέτρους μέχρι να έχουμε τη δυνατότητα να οπτικοποιήσουμε και να αναλύσουμε ποιοτικά τα αποτελέσματα. Ξεκινήστε με ένα χάρτη 10 x 10, 100 epochs training και ένα υποσύνολο των ταινιών (π.χ. 2000). Χρησιμοποιήστε την `time` για να έχετε μια εικόνα των χρόνων εκπαίδευσης. 


## Best matching units

Μετά από κάθε εκπαίδευση αποθηκεύστε σε μια μεταβλητή τα best matching units (bmus) για κάθε ταινία. Τα bmus μας δείχνουν σε ποιο νευρώνα ανήκει η κάθε ταινία. **Προσοχή: η σύμβαση των συντεταγμένων των νευρώνων στη Somoclu είναι (στήλη, γραμμή) δηλαδή το ανάποδο από την Python**. Με χρήση της [np.unique](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html) (μια πολύ χρήσιμη συνάρτηση στην άσκηση) αποθηκεύστε τα μοναδικά best matching units και τους δείκτες τους (indices) προς τις ταινίες. 

Σημειώστε ότι μπορεί να έχετε λιγότερα μοναδικά bmus από αριθμό νευρώνων γιατί μπορεί σε κάποιους νευρώνες να μην έχουν ανατεθεί ταινίες. Ως αριθμό νευρώνα θα θεωρήσουμε τον αριθμό γραμμής στον πίνακα μοναδικών bmus.



## Ομαδοποίηση (clustering)

Τυπικά, η ομαδοποίηση σε ένα χάρτη SOM προκύπτει από το unified distance matrix (U-matrix): για κάθε κόμβο υπολογίζεται η μέση απόστασή του από τους γειτονικούς κόμβους. Εάν χρησιμοποιηθεί μπλε χρώμα στις περιοχές του χάρτη όπου η τιμή αυτή είναι χαμηλή (μικρή απόσταση) και κόκκινο εκεί που η τιμή είναι υψηλή (μεγάλη απόσταση), τότε μπορούμε να πούμε ότι οι μπλε περιοχές αποτελούν clusters και οι κόκκινες αποτελούν σύνορα μεταξύ clusters.

To somoclu δίνει την επιπρόσθετη δυνατότητα να κάνουμε ομαδοποίηση των νευρώνων χρησιμοποιώντας οποιονδήποτε αλγόριθμο ομαδοποίησης του scikit-learn. Στην άσκηση θα χρησιμοποιήσουμε τον k-Means. Για τον αρχικό σας χάρτη δοκιμάστε ένα k=20 ή 25. Οι δύο προσεγγίσεις ομαδοποίησης είναι διαφορετικές, οπότε περιμένουμε τα αποτελέσματα να είναι κοντά αλλά όχι τα ίδια.



## Αποθήκευση του SOM

Επειδή η αρχικοποίηση του SOM γίνεται τυχαία και το clustering είναι και αυτό στοχαστική διαδικασία, οι θέσεις και οι ετικέτες των νευρώνων και των clusters θα είναι διαφορετικές κάθε φορά που τρέχετε τον χάρτη, ακόμα και με τις ίδιες παραμέτρους. Για να αποθηκεύσετε ένα συγκεκριμένο som και clustering χρησιμοποιήστε και πάλι την `joblib`. Μετά την ανάκληση ενός SOM θυμηθείτε να ακολουθήσετε τη διαδικασία για τα bmus.



## Οπτικοποίηση U-matrix, clustering και μέγεθος clusters

Για την εκτύπωση του U-matrix χρησιμοποιήστε τη `view_umatrix` με ορίσματα `bestmatches=True` και `figsize=(15, 15)` ή `figsize=(20, 20)`. Τα διαφορετικά χρώματα που εμφανίζονται στους κόμβους αντιπροσωπεύουν τα διαφορετικά clusters που προκύπτουν από τον k-Means. Μπορείτε να εμφανίσετε τη λεζάντα του U-matrix με το όρισμα `colorbar`. Μην τυπώνετε τις ετικέτες (labels) των δειγμάτων, είναι πολύ μεγάλος ο αριθμός τους.

Για μια δεύτερη πιο ξεκάθαρη οπτικοποίηση του clustering τυπώστε απευθείας τη μεταβλητή `clusters`.

Τέλος, χρησιμοποιώντας πάλι την `np.unique` (με διαφορετικό όρισμα) και την `np.argsort` (υπάρχουν και άλλοι τρόποι υλοποίησης) εκτυπώστε τις ετικέτες των clusters (αριθμοί από 0 έως k-1) και τον αριθμό των νευρώνων σε κάθε cluster, με φθίνουσα ή αύξουσα σειρά ως προς τον αριθμό των νευρώνων. Ουσιαστικά είναι ένα εργαλείο για να βρίσκετε εύκολα τα μεγάλα και μικρά clusters. 

Ακολουθεί ένα μη βελτιστοποιημένο παράδειγμα για τις τρεις προηγούμενες εξόδους:

<img src="https://image.ibb.co/i0tsfR/umatrix_s.jpg" width="35%">
<img src="https://image.ibb.co/nLgHEm/clusters.png" width="35%">




## Σημασιολογική ερμηνεία των clusters

Προκειμένου να μελετήσουμε τις τοπολογικές ιδιότητες του SOM και το αν έχουν ενσωματώσει σημασιολογική πληροφορία για τις ταινίες διαμέσου της διανυσματικής αναπαράστασης του tf-idf, των εμφυτευμάτων και των κατηγοριών, χρειαζόμαστε ένα κριτήριο ποιοτικής επισκόπησης των clusters. 

Θα υλοποιήσουμε το εξής κριτήριο: Λαμβάνουμε όρισμα έναν αριθμό (ετικέτα) cluster. Για το cluster αυτό βρίσκουμε όλους τους νευρώνες που του έχουν ανατεθεί από τον k-Means. Για όλους τους νευρώνες αυτούς βρίσκουμε όλες τις ταινίες που τους έχουν ανατεθεί (για τις οποίες αποτελούν bmus). Για όλες αυτές τις ταινίες τυπώνουμε ταξινομημένη τη συνολική στατιστική όλων των ειδών (κατηγοριών) και τις συχνότητές τους. Αν το cluster διαθέτει καλή συνοχή και εξειδίκευση, θα πρέπει κάποιες κατηγορίες να έχουν σαφώς μεγαλύτερη συχνότητα από τις υπόλοιπες. Θα μπορούμε τότε να αναθέσουμε αυτήν/ές την/τις κατηγορία/ες ως ετικέτες κινηματογραφικού είδους στο cluster.

Μπορείτε να υλοποιήσετε τη συνάρτηση αυτή όπως θέλετε. Μια πιθανή διαδικασία θα μπορούσε να είναι η ακόλουθη:

1. Ορίζουμε συνάρτηση `print_categories_stats` που δέχεται ως είσοδο λίστα με ids ταινιών. Δημιουργούμε μια κενή λίστα συνολικών κατηγοριών. Στη συνέχεια, για κάθε ταινία επεξεργαζόμαστε το string `categories` ως εξής: δημιουργούμε μια λίστα διαχωρίζοντας το string κατάλληλα με την `split` και αφαιρούμε τα whitespaces μεταξύ ετικετών με την `strip`. Προσθέτουμε τη λίστα αυτή στη συνολική λίστα κατηγοριών με την `extend`. Τέλος χρησιμοποιούμε πάλι την `np.unique` για να μετρήσουμε συχνότητα μοναδικών ετικετών κατηγοριών και ταξινομούμε με την `np.argsort`. Τυπώνουμε τις κατηγορίες και τις συχνότητες εμφάνισης ταξινομημένα. Χρήσιμες μπορεί να σας φανούν και οι `np.ravel`, `np.nditer`, `np.array2string` και `zip`.

2. Ορίζουμε τη βασική μας συνάρτηση `print_cluster_neurons_movies_report` που δέχεται ως όρισμα τον αριθμό ενός cluster. Με τη χρήση της `np.where` μπορούμε να βρούμε τις συντεταγμένες των bmus που αντιστοιχούν στο cluster και με την `column_stack` να φτιάξουμε έναν πίνακα bmus για το cluster. Προσοχή στη σειρά (στήλη - σειρά) στον πίνακα bmus. Για κάθε bmu αυτού του πίνακα ελέγχουμε αν υπάρχει στον πίνακα μοναδικών bmus που έχουμε υπολογίσει στην αρχή συνολικά και αν ναι προσθέτουμε το αντίστοιχο index του νευρώνα σε μια λίστα. Χρήσιμες μπορεί να είναι και οι `np.rollaxis`, `np.append`, `np.asscalar`. Επίσης πιθανώς να πρέπει να υλοποιήσετε ένα κριτήριο ομοιότητας μεταξύ ενός bmu και ενός μοναδικού bmu από τον αρχικό πίνακα bmus.

3. Υλοποιούμε μια βοηθητική συνάρτηση `neuron_movies_report`. Λαμβάνει ένα σύνολο νευρώνων από την `print_cluster_neurons_movies_report` και μέσω της `indices` φτιάχνει μια λίστα με το σύνολο ταινιών που ανήκουν σε αυτούς τους νευρώνες. Στο τέλος καλεί με αυτή τη λίστα την `print_categories_stats` που τυπώνει τις στατιστικές των κατηγοριών.

Μπορείτε βέβαια να προσθέσετε οποιαδήποτε επιπλέον έξοδο σας βοηθάει. Μια χρήσιμη έξοδος είναι πόσοι νευρώνες ανήκουν στο cluster και σε πόσους και ποιους από αυτούς έχουν ανατεθεί ταινίες.

Θα επιτελούμε τη σημασιολογική ερμηνεία του χάρτη καλώντας την `print_cluster_neurons_movies_report` με τον αριθμός ενός cluster που μας ενδιαφέρει. 

Παράδειγμα εξόδου για ένα cluster (μη βελτιστοποιημένος χάρτης, ωστόσο βλέπετε ότι οι μεγάλες κατηγορίες έχουν σημασιολογική  συνάφεια):

```
Overall Cluster Genres stats:  
[('"Horror"', 86), ('"Science Fiction"', 24), ('"B-movie"', 16), ('"Monster movie"', 10), ('"Creature Film"', 10), ('"Indie"', 9), ('"Zombie Film"', 9), ('"Slasher"', 8), ('"World cinema"', 8), ('"Sci-Fi Horror"', 7), ('"Natural horror films"', 6), ('"Supernatural"', 6), ('"Thriller"', 6), ('"Cult"', 5), ('"Black-and-white"', 5), ('"Japanese Movies"', 4), ('"Short Film"', 3), ('"Drama"', 3), ('"Psychological thriller"', 3), ('"Crime Fiction"', 3), ('"Monster"', 3), ('"Comedy"', 2), ('"Western"', 2), ('"Horror Comedy"', 2), ('"Archaeology"', 2), ('"Alien Film"', 2), ('"Teen"', 2), ('"Mystery"', 2), ('"Adventure"', 2), ('"Comedy film"', 2), ('"Combat Films"', 1), ('"Chinese Movies"', 1), ('"Action/Adventure"', 1), ('"Gothic Film"', 1), ('"Costume drama"', 1), ('"Disaster"', 1), ('"Docudrama"', 1), ('"Film adaptation"', 1), ('"Film noir"', 1), ('"Parody"', 1), ('"Period piece"', 1), ('"Action"', 1)]```
   


## Tips για το SOM και το clustering

- Για την ομαδοποίηση ένα U-matrix καλό είναι να εμφανίζει και μπλε-πράσινες περιοχές (clusters) και κόκκινες περιοχές (ορίων). Παρατηρήστε ποια σχέση υπάρχει μεταξύ αριθμού ταινιών στο final set, μεγέθους grid και ποιότητας U-matrix.
- Για το k του k-Means προσπαθήστε να προσεγγίζει σχετικά τα clusters του U-matrix (όπως είπαμε είναι διαφορετικοί μέθοδοι clustering). Μικρός αριθμός k δεν θα σέβεται τα όρια. Μεγάλος αριθμός θα δημιουργεί υπο-clusters εντός των clusters που φαίνονται στο U-matrix. Το τελευταίο δεν είναι απαραίτητα κακό, αλλά μεγαλώνει τον αριθμό clusters που πρέπει να αναλυθούν σημασιολογικά.
- Σε μικρούς χάρτες και με μικρά final sets δοκιμάστε διαφορετικές παραμέτρους για την εκπαίδευση του SOM. Σημειώστε τυχόν παραμέτρους που επηρεάζουν την ποιότητα του clustering για το dataset σας ώστε να τις εφαρμόσετε στους μεγάλους χάρτες.
- Κάποια τοπολογικά χαρακτηριστικά εμφανίζονται ήδη σε μικρούς χάρτες. Κάποια άλλα χρειάζονται μεγαλύτερους χάρτες. Δοκιμάστε μεγέθη 20x20, 25x25 ή και 30x30 και αντίστοιχη προσαρμογή των k. Όσο μεγαλώνουν οι χάρτες, μεγαλώνει η ανάλυση του χάρτη αλλά μεγαλώνει και ο αριθμός clusters που πρέπει να αναλυθούν.




## Ανάλυση τοπολογικών ιδιοτήτων χάρτη SOM

Μετά το πέρας της εκπαίδευσης και του clustering θα έχετε ένα χάρτη με τοπολογικές ιδιότητες ως προς τα είδη των ταίνιών της συλλογής σας, κάτι αντίστοιχο με την εικόνα στην αρχή της Εφαρμογής 2 αυτού του notebook. Η συγκεκριμένη εικόνα είναι μόνο για εικονογράφιση, δεν είναι χάρτης SOM καιδεν έχει καμία σχέση με τη συλλογή δεδομένων και τις κατηγορίες μας.

Για τον τελικό χάρτη SOM που θα παράξετε για τη συλλογή σας, αναλύστε σε markdown με συγκεκριμένη αναφορά σε αριθμούς clusters και τη σημασιολογική ερμηνεία τους τις εξής τρεις τοπολογικές ιδιότητες του SOM: 

1. Δεδομένα που έχουν μεγαλύτερη πυκνότητα πιθανότητας στο χώρο εισόδου τείνουν να απεικονίζονται με περισσότερους νευρώνες στο χώρο μειωμένης διαστατικότητας. Δώστε παραδείγματα από συχνές και λιγότερο συχνές κατηγορίες ταινιών. Χρησιμοποιήστε τις στατιστικές των κατηγοριών στη συλλογή σας και τον αριθμό κόμβων που χαρακτηρίζουν.
2. Μακρινά πρότυπα εισόδου τείνουν να απεικονίζονται απομακρυσμένα στο χάρτη. Υπάρχουν χαρακτηριστικές κατηγορίες ταινιών που ήδη από μικρούς χάρτες τείνουν να τοποθετούνται σε διαφορετικά ή απομονωμένα σημεία του χάρτη.
3. Κοντινά πρότυπα εισόδου τείνουν να απεικονίζονται κοντά στο χάρτη. Σε μεγάλους χάρτες εντοπίστε είδη ταινιών και κοντινά τους υποείδη.

Προφανώς τοποθέτηση σε 2 διαστάσεις που να σέβεται μια απόλυτη τοπολογία δεν είναι εφικτή, αφενός γιατί δεν υπάρχει κάποια απόλυτη εξ ορισμού για τα κινηματογραφικά είδη ακόμα και σε πολλές διαστάσεις, αφετέρου γιατί πραγματοποιούμε μείωση διαστατικότητας.

Εντοπίστε μεγάλα clusters και μικρά clusters που δεν έχουν σαφή χαρακτηριστικά. Εντοπίστε clusters συγκεκριμένων ειδών που μοιάζουν να μην έχουν τοπολογική συνάφεια με γύρω περιοχές. Προτείνετε πιθανές ερμηνείες.


Τέλος, εντοπίστε clusters που έχουν κατά την άποψή σας ιδιαίτερο ενδιαφέρον στη συλλογή της ομάδας σας (data exploration / discovery value) και σχολιάστε.



# Τελική παράδοση άσκησης

- Θα παραδώσετε στο helios το παρόν notebook επεξεργασμένο ή ένα ή δύο νέα zipαρισμένα με τις απαντήσεις σας για τα ζητούμενα και των δύο εφαρμογών. 
- Θυμηθείτε ότι η ανάλυση του χάρτη στο markdown με αναφορά σε αριθμούς clusters πρέπει να αναφέρεται στον τελικό χάρτη με τα κελιά ορατά που θα παραδώσετε αλλιώς ο χάρτης που θα προκύψει θα είναι διαφορετικός και τα labels των clusters δεν θα αντιστοιχούν στην ανάλυσή σας. 
- Μην ξεχάσετε στην αρχή ένα κελί markdown με **τα στοιχεία της ομάδας σας**.

<table>
  <tr><td align="center">
    <font size="4">Παρακαλούμε διατρέξτε βήμα-βήμα το notebook για να μην ξεχάσετε παραδοτέα</font>
</td>
  </tr>
</table>